In [1]:
from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2

In [2]:
def detect(gray, frame):
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
    return frame

In [3]:
# Returns EAR given eye landmarks
def eye_aspect_ratio(eye):
    # Compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])

    # Compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = np.linalg.norm(eye[0] - eye[3])

    # Compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)

    # Return the eye aspect ratio
    return ear

In [4]:
# Returns MAR given eye landmarks
def mouth_aspect_ratio(mouth):
    # Compute the euclidean distances between the three sets
    # of vertical mouth landmarks (x, y)-coordinates
    A = np.linalg.norm(mouth[13] - mouth[19])
    B = np.linalg.norm(mouth[14] - mouth[18])
    C = np.linalg.norm(mouth[15] - mouth[17])

    # Compute the euclidean distance between the horizontal
    # mouth landmarks (x, y)-coordinates
    D = np.linalg.norm(mouth[12] - mouth[16])

    # Compute the mouth aspect ratio
    mar = (A + B + C) / (2 * D)

    # Return the mouth aspect ratio
    return mar

In [5]:
# Thresholds and consecutive frame length for triggering action.
MOUTH_AR_THRESH = 0.18
MOUTH_AR_CONSECUTIVE_FRAMES = 4
EYE_AR_THRESH = 0.17
EYE_AR_CONSECUTIVE_FRAMES = 4
WINK_AR_DIFF_THRESH = 0.04
WINK_AR_CLOSE_THRESH = 0.19
WINK_CONSECUTIVE_FRAMES = 3

In [6]:
# Initialize the frame counters for each action as well as 
# booleans used to indicate if action is performed or not
stop_counter = 0
left_counter = 0
right_counter = 0
forward_counter=0
anchor_point = (0, 0)

In [7]:
WHITE_COLOR = (255, 255, 255)
YELLOW_COLOR = (0, 255, 255)
RED_COLOR = (0, 0, 255)
GREEN_COLOR = (0, 255, 0)
BLUE_COLOR = (255, 0, 0)
BLACK_COLOR = (0, 0, 0)

In [8]:
font = cv2.FONT_HERSHEY_SIMPLEX
positionOfText = (5,100)
fontScale = 1
lineType = 2
fontColor = GREEN_COLOR

In [9]:
# Initialize Dlib's face detector and then create
# the facial landmark predictor
p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

In [10]:
# Grab the indexes of the facial landmarks for the left and
# right eye, nose and mouth respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
(nStart, nEnd) = face_utils.FACIAL_LANDMARKS_IDXS["nose"]
(mStart, mEnd) = face_utils.FACIAL_LANDMARKS_IDXS["mouth"]

In [11]:
cap = cv2.VideoCapture(0)
resolution_w = 1366
resolution_h = 768
cam_w = 640
cam_h = 480
unit_w = resolution_w / cam_w
unit_h = resolution_h / cam_h


while True:
    
    # load the input image and convert it to grayscale
    _, frame = cap.read()
    # detect faces in the grayscale image
    frame = imutils.resize(frame, width=cam_w, height=cam_h)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    rects = detector(gray, 0)
    
    if len(rects)>0:
        rect=rects[0]
        
    else:
        cv2.imshow("Output", frame)
        key = cv2.waitKey(1) & 0xFF
        continue
    
    # Determine the facial landmarks for the face region, then
    # convert the facial landmark (x, y)-coordinates to a NumPy array
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
    
    for (x, y) in shape:
            cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
    
    mouth = shape[mStart:mEnd]
    leftEye = shape[lStart:lEnd]
    rightEye = shape[rStart:rEnd]
    nose = shape[nStart:nEnd]
    

    # Average the mouth aspect ratio together for both eyes
    mar = mouth_aspect_ratio(mouth)
    leftEAR = eye_aspect_ratio(leftEye)
    rightEAR = eye_aspect_ratio(rightEye)
    ear = (leftEAR + rightEAR) / 2.0
    diff_ear = np.abs(leftEAR - rightEAR)
    
    
    # Compute the convex hull for the left and right eye, then
    # visualize each of the eyes
    mouthHull = cv2.convexHull(mouth)
    leftEyeHull = cv2.convexHull(leftEye)
    rightEyeHull = cv2.convexHull(rightEye)
    cv2.drawContours(frame, [mouthHull], -1, YELLOW_COLOR, 1)
    cv2.drawContours(frame, [leftEyeHull], -1, YELLOW_COLOR, 1)
    cv2.drawContours(frame, [rightEyeHull], -1, YELLOW_COLOR, 1)
        
        
        
    for (x, y) in np.concatenate((mouth, leftEye, rightEye), axis=0):
        cv2.circle(frame, (x, y), 2, GREEN_COLOR, -1)
        
    # Check to see if the eye aspect ratio is below the blink
    # threshold, and if so, increment the blink frame counter
    # print("Code here")
    # print(diff_ear,WINK_AR_DIFF_THRESH)
    
    #print('mar value is' ,mar)
    
    if mar > MOUTH_AR_THRESH:
        stop_counter = stop_counter + 1
        #print('stop counter value is',stop_counter)
        if stop_counter > MOUTH_AR_CONSECUTIVE_FRAMES:
            print('break applied')
            
            cv2.putText(frame,'break applied', positionOfText, font, fontScale,fontColor,lineType)
            stop_counter = 0
    
    if diff_ear > WINK_AR_DIFF_THRESH:
        #print(leftEAR,rightEAR)
        if leftEAR < rightEAR:
            if leftEAR < EYE_AR_THRESH:
                left_counter = left_counter + 1
                
                #print('left counter value is',left_counter)
                if left_counter > WINK_CONSECUTIVE_FRAMES:
                    print('move left')
                    
                    cv2.putText(frame,'turn left', positionOfText, font, fontScale,fontColor,lineType)
                    left_counter = 0

        elif leftEAR > rightEAR:
            # print("Shown Below ",rightEAR)
            if rightEAR < EYE_AR_THRESH:
                right_counter = right_counter+1
                   
                #print('right counter value is',right_counter)
                if right_counter > WINK_CONSECUTIVE_FRAMES:
                    print('move right')
                    
                    cv2.putText(frame,'turn right', positionOfText, font, fontScale,fontColor,lineType)
                    right_counter = 0

    else:
      if ear <= EYE_AR_THRESH:
            forward_counter = forward_counter+ 1
            
            #print('forward counter value is',forward_counter)
            if forward_counter > EYE_AR_CONSECUTIVE_FRAMES:
                print('move forward')
                
                cv2.putText(frame,'move forward', positionOfText, font, fontScale,fontColor,lineType)
                forward_counter = 0
                
    
#     print("Image Shown Here",frame)
    # show the output image with the face detections + facial landmarks
    cv2.imshow("Output", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 

cv2.destroyAllWindows()
cap.release()

break applied
break applied
break applied
break applied
